# Dataset Builder

Takes raw data and makes a real dataset ready for preprocessing.

## Importing

In [ ]:
import pandas as pd

In [ ]:
# Get Dataset into pandas dataframe
df_pesagens = pd.read_csv('../data/raw/pesagens2014.csv', sep=',')
df_rotas = pd.read_csv('../data/raw/rotas.csv', sep=';')

## Info about the raw data

In [ ]:
df_pesagens.info()

In [ ]:
df_rotas.info()

In [ ]:
df_pesagens.head()

In [ ]:
df_rotas.head()

By looking at both data, as well as looking at the [documentation of the data](http://dados.recife.pe.gov.br/dataset/pesagem-de-coletas-de-residuos), we know that the point of connection for both data is `ROTA_ID` attribute. 

We've gotta be sure that all instances from `pesagens` dataset has a valid `ROTA_ID` that is present on `roteirizacao` dataset, for this, we'll aggregate all valid data and non-valid data will be discarded.

## Merging Rotas Data into Pesagens 

Now we're gonna merge both datasets into just one dataset. This is achieved by using the `merge` method from **pandas**, but, as seen in our tests, if we have duplicate indexes we may see a duplication of our data. 

To fix this, we're gonna drop duplicate indexes present on our data, to make sure the merge is successfull and  no duplicate data is present in the merge.

In [ ]:
df_pesagens = df_pesagens.reset_index().drop_duplicates(subset='PES_ID', keep='first')
df_rotas = df_rotas.reset_index().drop_duplicates(subset='ROTA_ID', keep='first')

Duplicates removed, we can now merge into one dataset.

In [ ]:
df = df_pesagens.merge(df_rotas, on='ROTA_ID')

There's a few data is duplicated due to the merge, that is:

- `EMP_ID_x` and `EMP_ID_y`
- `_id` and `index`

We'll remove the duplicated data and keep only the attributes that stores real data.

In [ ]:
df.rename(columns={'EMP_ID_x': 'EMP_ID'}, inplace=True)
df.drop(columns=['EMP_ID_y'], inplace=True)


In [ ]:
df.drop(['index_x', 'index_y'], axis=1, inplace=True)
df.drop(['_id'], axis=1, inplace=True)

## A few adaptations

Altough the dataset is merged there's a few data that needs adaptation. We have identified the following inconsistencies with the data:

- `PES_DATAINI` and `PES_DATAFIM` contains only date, not datetime format
- `PES_HRINI` and `PES_HRFIM` contains only time data, not datetime format

Since **datetime** format combines both information, we'll combine those data into `DATETIME_INI` and `DATETIME_FIM`.

In [ ]:
df['PES_DATAINI'] = df['PES_DATAINI'].apply(lambda x: x[:10])
df['PES_DATAFIM'] = df['PES_DATAFIM'].apply(lambda x: x[:10])

df['PES_HRINI'] = df['PES_HRINI'].apply(lambda x: x[11:])
df['PES_HRFIM'] = df['PES_HRFIM'].apply(lambda x: x[11:])

df['DATETIME_INI'] = pd.to_datetime(df['PES_DATAINI'] + ' ' + df['PES_HRINI'])
df['DATETIME_FIM'] = pd.to_datetime(df['PES_DATAFIM'] + ' ' + df['PES_HRFIM'])

df.drop(['PES_DATAINI', 'PES_DATAFIM', 'PES_HRINI', 'PES_HRFIM'], axis=1, inplace=True)

## Exporting our dataset

Simply we're gonna export into a `.csv`.

In [ ]:
df.to_csv('../data/dataset.csv', sep=',', index=False)